# Instalando as bibliotecas usadas

Algumas das bibliotecas usadas não fazem parte da distribuição padrão do Python.

A célula abaixo realiza a instalação delas para garantir o funcionamento do código.

In [ ]:
!pip install pandas
!pip install requests

# Questão 1) Coleta de dados

## Baixando os microdados do ENADE

O código utiliza a biblioteca ```requests```, mas a primeira célula já executa o comando ```!pip install requests```.

In [2]:
# Script para download de arquivos do Google Drive encontrado no Stack Overflow
# https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url

import requests
import os

def download_file(id, destination):
    URL = "https://drive.google.com/uc?id=15TygyU78fPtyMDwnwGTQGNdpXtvCZ6RQ&export=download"

    session = requests.Session()
    response = session.get(URL, params={'id': id})
    token = get_confirm_token(response)
    if(token):
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params)
    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

# https://stackoverflow.com/questions/30582162/creating-a-missing-directory-file-structure-python
def createAndOpen(filename, mode):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    return open(filename, mode)

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with createAndOpen(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

# Endereço do arquivo CSV
endereco_arquivo_zip = os.path.join(os.getcwd(), 'microdados_ENADE.zip')

if __name__ == "__main__":
    file_id = '15TygyU78fPtyMDwnwGTQGNdpXtvCZ6RQ'
    destination = endereco_arquivo_zip
    download_file(file_id, destination)

Agora irei extrair o arquivo .zip que foram baixados

In [3]:
import zipfile

if __name__ == '__main__':
    with zipfile.ZipFile(endereco_arquivo_zip, 'r') as zip_ref:
        zip_ref.extractall() # Como não forneci nenhum endereço, vai extrair na pasta atual

Os arquivos com os dados vieram em formato.txt, mas o nó CSV Reader do KNIME consege lê-los sem problema, então não há necessidade de converter o formato do arquivo pra .csv.

Agora irei mover os arquivos CSV para a pasta Downloads

In [ ]:
import shutil

# Verifica se o arquivo existe ou não, se existe deleta
# Isso é feito para o método shutil.move() conseguir transferir o arquivo
def verificaArquivoExiste(file_path):
    if (os.path.isfile(file_path)):
        os.remove(file_path)

download_csv_2017_path = os.path.join(os.getenv('USERPROFILE'), 'Downloads', 'MICRODADOS_ENADE_2017.txt')
download_csv_2018_path = os.path.join(os.getenv('USERPROFILE'), 'Downloads', 'MICRODADOS_ENADE_2018.txt')
download_csv_2019_path = os.path.join(os.getenv('USERPROFILE'), 'Downloads', 'MICRODADOS_ENADE_2019.txt')

verificaArquivoExiste(download_csv_2017_path)
verificaArquivoExiste(download_csv_2018_path)
verificaArquivoExiste(download_csv_2019_path)

path_download_2017 = os.path.join(os.getcwd(), 'Microdados ENADE', '2017', '3.DADOS', 'MICRODADOS_ENADE_2017.txt')
path_download_2018 = os.path.join(os.getcwd(), 'Microdados ENADE', '2018', '3.DADOS', 'MICRODADOS_ENADE_2018.txt')
path_download_2019 = os.path.join(os.getcwd(), 'Microdados ENADE', '2019', '3.DADOS', 'MICRODADOS_ENADE_2019.txt')

# Faço uma cópia para a pasta local Downloads do Windows
shutil.copy(path_download_2017, os.path.join(os.getenv('USERPROFILE'), 'Downloads'))
shutil.copy(path_download_2018, os.path.join(os.getenv('USERPROFILE'), 'Downloads'))
shutil.copy(path_download_2019, os.path.join(os.getenv('USERPROFILE'), 'Downloads'))


# Crio uma pasta para armazenar uma cópia local dos arquivos CSV para facilitar o acesso a eles
os.makedirs('Microdados arquivos CSV', exist_ok = True)

# Agora faço uma cópia local, para caso o usuário queira ver os dados manualmente
path_local_2017 = os.path.join(os.getcwd(), 'Microdados arquivos CSV', 'Microdados_ENADE_2017.csv')
path_local_2018 = os.path.join(os.getcwd(), 'Microdados arquivos CSV', 'Microdados_ENADE_2018.csv')
path_local_2019 = os.path.join(os.getcwd(), 'Microdados arquivos CSV', 'Microdados_ENADE_2019.csv')

shutil.copy(path_download_2017, os.path.join(os.getenv('USERPROFILE'), os.path.join(os.getcwd(), path_local_2017)))
shutil.copy(path_download_2018, os.path.join(os.getenv('USERPROFILE'), os.path.join(os.getcwd(), path_local_2018)))
shutil.copy(path_download_2019, os.path.join(os.getenv('USERPROFILE'), os.path.join(os.getcwd(), path_local_2019)))

# Questão 3) Cria o banco de dados

Primeiro tenho que criar a conexão com o BD

In [5]:
import sqlite3
from sqlite3 import Error
import os

# https://www.sqlitetutorial.net/sqlite-python/

def createConnection(database_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(database_file)
        return conn
    except Error as e:
        print(e)

def getDatabasePath():
    return os.path.join(os.getcwd(), 'DW_Prova_BD.db')

if __name__ == '__main__':
    conn = createConnection(getDatabasePath())

## Cria as tabelas

In [6]:
def criaTabelaSQL(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [7]:
def main():
    db = getDatabasePath()

    # Vou criar a tabela fato
    SQL_cria_Fato_ENADE = """ CREATE TABLE IF NOT EXISTS Fato_ENADE (
        fk_tempo_id integer,
        fk_info_es_curso_id integer,
        fk_estudante_id integer,
        fk_num_item_id integer,
        fk_vetor_id integer,
        fk_presenca_id integer,
        fk_tp_sit_q_disc_id integer,
        fk_nt_fg_ce_id integer,
        fk_percepcao_prova_id integer,
        fk_questionario_estudante_id integer,
        FOREIGN KEY (fk_tempo_id) REFERENCES Dimensao_Tempo(tempo_id),
        FOREIGN KEY (fk_info_es_curso_id) REFERENCES Dimensao_INFO_ES_CURSO(info_es_curso_id),
        FOREIGN KEY (fk_estudante_id) REFERENCES Dimensao_ESTUDANTE(estudante_id),
        FOREIGN KEY (fk_num_item_id) REFERENCES Dimensao_NU_ITEM(nu_item_id),
        FOREIGN KEY (fk_vetor_id) REFERENCES Dimensao_VETORES(vetor_id),
        FOREIGN KEY (fk_presenca_id) REFERENCES Dimensao_TP_PRESENCA(tp_pres_id),
        FOREIGN KEY (fk_tp_sit_q_disc_id) REFERENCES Dimensao_TP_SIT_Q_DISC(tp_sit_q_disc_id),
        FOREIGN KEY (fk_nt_fg_ce_id) REFERENCES Dimensao_NT_FG_CE(nt_fg_ce_id),
        FOREIGN KEY (fk_percepcao_prova_id) REFERENCES Dimensao_PERCEPCAO_PROVA(percepcao_prova_id),
        FOREIGN KEY (fk_questionario_estudante_id) REFERENCES Dimensao_QUESTIONARIO_ESTUDANTE(questionario_estudante_id),
        PRIMARY KEY (fk_tempo_id, fk_info_es_curso_id,
                        fk_estudante_id, fk_num_item_id,
                        fk_vetor_id,fk_presenca_id,
                        fk_tp_sit_q_disc_id, fk_nt_fg_ce_id,
                        fk_percepcao_prova_id, fk_questionario_estudante_id)
    );"""

    # Vou criar as tabelas dimensão
    ## Tabela TEMPO
    SQL_cria_Dimensao_TEMPO = """ CREATE TABLE IF NOT EXISTS Dimensao_TEMPO (
        tempo_id integer PRIMARY KEY AUTOINCREMENT,
        nu_ano integer,
        ano_fim_em integer,
        ano_in_grad integer
    );"""

    ## Tabela INFO_ES_CURSO
    SQL_cria_Dimensao_INFO_ES_CURSO = """ CREATE TABLE IF NOT EXISTS Dimensao_INFO_ES_CURSO (
        info_es_curso_id integer PRIMARY KEY AUTOINCREMENT,
        co_ies integer,
        co_categad integer,
        co_orgacad integer,
        co_grupo integer,
        co_curso integer,
        co_modalidade integer,
        co_munic_curso integer,
        co_uf_curso integer,
        co_regiao_curso integer
    );"""

    ## Tabela ESTUDANTE
    SQL_cria_Dimensao_ESTUDANTE = """ CREATE TABLE IF NOT EXISTS Dimensao_ESTUDANTE (
        estudante_id integer PRIMARY KEY AUTOINCREMENT,
        nu_idade integer,
        tp_sexo string,
        co_turno_graduacao integer,
        tp_inscricao_adm integer,
        tp_inscricao integer
    );"""

    ## Tabela NU_ITEM
    SQL_cria_Dimensao_NU_ITEM = """ CREATE TABLE IF NOT EXISTS Dimensao_NU_ITEM (
        nu_item_id integer PRIMARY KEY AUTOINCREMENT,
        nu_item_ofg integer,
        nu_item_ofg_z integer,
        nu_item_ofg_x integer,
        nu_item_ofg_n integer,
        nu_item_oce integer,
        nu_item_oce_z integer,
        nu_item_oce_x integer,
        nu_item_oce_n integer
    );"""

    ## Tabela VETORES
    SQL_cria_Dimensao_VETORES = """ CREATE TABLE IF NOT EXISTS Dimensao_VETORES (
        vetor_id integer PRIMARY KEY AUTOINCREMENT,
        ds_vt_gab_ofg_orig string,
        ds_vt_gab_ofg_fin string,
        ds_vt_gab_oce_orig string,
        ds_vt_gab_oce_fin string,
        ds_vt_esc_ofg string,
        ds_vt_ace_ofg integer,
        ds_vt_esc_oce string,
        ds_vt_ace_oce real
    );"""

    ## Tabela TP_PRES
    SQL_cria_Dimensao_TP_PRES = """ CREATE TABLE IF NOT EXISTS Dimensao_TP_PRESENCA (
        tp_pres_id integer PRIMARY KEY AUTOINCREMENT,
        tp_pres integer,
        tp_pr_ger integer,
        tp_pr_ob_fg integer,
        tp_pr_di_fg integer,
        tp_pr_ob_ce integer,
        tp_pr_di_ce integer
    );"""

    ## Tabela TP_SIT_Q_DISC
    SQL_cria_Dimensao_TP_SIT_Q_DISC = """ CREATE TABLE IF NOT EXISTS Dimensao_TP_SIT_Q_DISC (
        tp_sit_q_disc_id integer PRIMARY KEY AUTOINCREMENT,
        tp_sfg_d1 integer,
        tp_sfg_d2 integer,
        tp_sce_d1 integer,
        tp_sce_d2 integer,
        tp_sce_d3 integer
    );"""

    ## Tabela NT_FG_CE
    SQL_cria_Dimensao_NT_FG_CE = """ CREATE TABLE IF NOT EXISTS Dimensao_NT_FG_CE (
        nt_fg_ce_id integer PRIMARY KEY AUTOINCREMENT,
        fk_nt_fg_id integer,
        fk_nt_ce_id integer,
        FOREIGN KEY (fk_nt_fg_id) REFERENCES Dimensao_NT_FG(nt_fg_id),
        FOREIGN KEY (fk_nt_ce_id) REFERENCES Dimensao_NT_CE(nt_ce_id)
    );"""

    ## Tabela NT_FG
    SQL_cria_Dimensao_NT_FG = """ CREATE TABLE IF NOT EXISTS Dimensao_NT_FG (
        nt_fg_id integer PRIMARY KEY AUTOINCREMENT,
        nt_ger string,
        nt_fg string,
        nt_obj_fg string,
        nt_dis_fg string,
        nt_fg_d1 integer,
        nt_fg_d1_pt integer,
        nt_fg_d1_ct integer,
        nt_fg_d2 integer,
        nt_fg_d2_pt integer,
        nt_fg_d2_ct integer
    );"""

    ## Tabela NT_CE
    SQL_cria_Dimensao_NT_CE = """ CREATE TABLE IF NOT EXISTS Dimensao_NT_CE (
        nt_ce_id integer PRIMARY KEY AUTOINCREMENT,
        nt_ce string,
        nt_obj_ce string,
        nt_dis_ce string,
        nt_ce_d1 integer,
        nt_ce_d2 integer,
        nt_ce_d3 integer
    );"""

    ## Tabela PERCEPCAO_PROVA
    SQL_cria_Dimensao_PERCEPCAO_PROVA = """ CREATE TABLE IF NOT EXISTS Dimensao_PERCEPCAO_PROVA (
        percepcao_prova_id integer PRIMARY KEY AUTOINCREMENT,
        co_rs_i1 text,
        co_rs_i2 text,
        co_rs_i3 text,
        co_rs_i4 text,
        co_rs_i5 text,
        co_rs_i6 text,
        co_rs_i7 text,
        co_rs_i8 text,
        co_rs_i9 text
    );"""

    ## Tabela QUESTIONARIO_ESTUDANTE
    SQL_cria_Dimensao_QUESTIONARIO_ESTUDANTE = """ CREATE TABLE IF NOT EXISTS Dimensao_QUESTIONARIO_ESTUDANTE (
        questionario_estudante_id integer PRIMARY KEY AUTOINCREMENT,
        string_questionario_estudante string
    );"""

    # Crio a conexão com o banco de dados
    conn = createConnection(db)

    # Crio as tabelas no SQL
    if conn is not None:
        criaTabelaSQL(conn, SQL_cria_Fato_ENADE)
        criaTabelaSQL(conn, SQL_cria_Dimensao_TEMPO)
        criaTabelaSQL(conn, SQL_cria_Dimensao_INFO_ES_CURSO)
        criaTabelaSQL(conn, SQL_cria_Dimensao_ESTUDANTE)
        criaTabelaSQL(conn, SQL_cria_Dimensao_NU_ITEM)
        criaTabelaSQL(conn, SQL_cria_Dimensao_VETORES)
        criaTabelaSQL(conn, SQL_cria_Dimensao_TP_PRES)
        criaTabelaSQL(conn, SQL_cria_Dimensao_TP_SIT_Q_DISC)
        criaTabelaSQL(conn, SQL_cria_Dimensao_NT_FG_CE)
        criaTabelaSQL(conn, SQL_cria_Dimensao_NT_FG)
        criaTabelaSQL(conn, SQL_cria_Dimensao_NT_CE)
        criaTabelaSQL(conn, SQL_cria_Dimensao_PERCEPCAO_PROVA)
        criaTabelaSQL(conn, SQL_cria_Dimensao_QUESTIONARIO_ESTUDANTE)
    else:
        print("ERRO: Não foi possível criar a conexão com o banco de dados.")

In [8]:
if __name__ == '__main__':
    main()

# Questão 4) Carga de dados

## Insere dados nas tabelas

Função que insere os dados no banco

In [9]:
import pandas as pd

def insereDadosNoBD(df, table_name):
    conn = createConnection(getDatabasePath())
    df.to_sql(table_name, conn, if_exists='append', index=False)

Agora vou pegar as colunas usadas em cada tabela

In [10]:
# Colunas que serão usadas para cada tabela
# Os arrays de seleção de coluna são case sensitive, se colocar o nome da coluna com tudo minúsculo o SQLite não vai reconhecer
colunas_Dimensao_TEMPO = ['NU_ANO', 'ANO_FIM_EM', 'ANO_IN_GRAD']

colunas_Dimensao_INFO_ES_CURSO = ['CO_IES', 'CO_CATEGAD', 'CO_ORGACAD', 'CO_GRUPO', 'CO_CURSO', 'CO_MODALIDADE', 'CO_MUNIC_CURSO', 'CO_UF_CURSO', 'CO_REGIAO_CURSO']

colunas_Dimensao_ESTUDANTE = ['NU_IDADE', 'TP_SEXO', 'CO_TURNO_GRADUACAO', 'TP_INSCRICAO_ADM', 'TP_INSCRICAO']

colunas_Dimensao_NU_ITEM = ['NU_ITEM_OFG', 'NU_ITEM_OFG_Z', 'NU_ITEM_OFG_X', 'NU_ITEM_OFG_N', 'NU_ITEM_OCE', 'NU_ITEM_OCE_Z', 'NU_ITEM_OCE_X', 'NU_ITEM_OCE_N']

colunas_Dimensao_VETORES = ['DS_VT_GAB_OFG_ORIG', 'DS_VT_GAB_OFG_FIN', 'DS_VT_GAB_OCE_ORIG', 'DS_VT_GAB_OCE_FIN', 'DS_VT_ESC_OFG', 'DS_VT_ACE_OFG', 'DS_VT_ESC_OCE', 'DS_VT_ACE_OCE']

colunas_Dimensao_TP_PRES = ['TP_PRES', 'TP_PR_GER', 'TP_PR_OB_FG', 'TP_PR_DI_FG', 'TP_PR_OB_CE', 'TP_PR_DI_CE']

colunas_Dimensao_TP_SIT_Q_DISC = ['TP_SFG_D1', 'TP_SFG_D2', 'TP_SCE_D1', 'TP_SCE_D2', 'TP_SCE_D3']

colunas_Dimensao_NT_FG = ['NT_GER', 'NT_FG', 'NT_OBJ_FG', 'NT_DIS_FG', 'NT_FG_D1', 'NT_FG_D1_PT', 'NT_FG_D1_CT', 'NT_FG_D2', 'NT_FG_D2_PT', 'NT_FG_D2_CT']
colunas_Dimensao_NT_CE = ['NT_CE', 'NT_OBJ_CE', 'NT_DIS_CE', 'NT_CE_D1', 'NT_CE_D2', 'NT_CE_D3']

colunas_Dimensao_PERCEPCAO_PROVA = ['CO_RS_I1', 'CO_RS_I2', 'CO_RS_I3', 'CO_RS_I4', 'CO_RS_I5', 'CO_RS_I6', 'CO_RS_I7', 'CO_RS_I8', 'CO_RS_I9']

colunas_Dimensao_QUESTIONARIO_ESTUDANTE = ['']

Agora vou inserir os dados, usando as colunas definidas acima para cada tabela

In [ ]:
def insereTudoNoBD(array_array_colunas, array_tabelas_SQL):
    for i in range(len(array_array_colunas)):
        # Insere dados de 2017 na tabela
        df = pd.read_csv(path_local_2017, usecols = array_array_colunas[i], sep = ';')
        insereDadosNoBD(df, array_tabelas_SQL[i])

        # Insere dados de 2018 na tabela
        df = pd.read_csv(path_local_2018, usecols = array_array_colunas[i], sep = ';')
        insereDadosNoBD(df, array_tabelas_SQL[i])
        
        # Insere dados de 2019 na tabela
        df = pd.read_csv(path_local_2019, usecols = array_array_colunas[i], sep = ';')
        insereDadosNoBD(df, array_tabelas_SQL[i])

array_array_colunas = [colunas_Dimensao_TEMPO, colunas_Dimensao_INFO_ES_CURSO, colunas_Dimensao_ESTUDANTE,
    colunas_Dimensao_NU_ITEM, colunas_Dimensao_VETORES, colunas_Dimensao_TP_PRES,
    colunas_Dimensao_TP_SIT_Q_DISC, colunas_Dimensao_NT_FG, colunas_Dimensao_NT_CE,
    colunas_Dimensao_PERCEPCAO_PROVA]

array_tabelas_SQL = ['Dimensao_TEMPO', 'Dimensao_INFO_ES_CURSO', 'Dimensao_ESTUDANTE',
    'Dimensao_NU_ITEM', 'Dimensao_VETORES', 'Dimensao_TP_PRES',
    'Dimensao_TP_SIT_Q_DISC', 'Dimensao_NT_FG', 'Dimensao_NT_CE',
    'Dimensao_PERCEPCAO_PROVA']

# Não consegui fazer funcionar, então deixei como comentário
#array_array_colunas.append(colunas_Dimensao_QUESTIONARIO_ESTUDANTE)
#array_tabelas_SQL.append('Dimensao_QUESTIONARIO_ESTUDANTE')

insereTudoNoBD(array_array_colunas, array_tabelas_SQL)